In [15]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 18 20:13:09 2022

@author: Furqan
"""
import pandasql as ps
import pandas as pd
import psycopg2 as GP
import datetime
from datetime import  timedelta
import urllib

from sqlalchemy import create_engine
import hashlib
GP_host="135.50.134.37"
GP_dbname='gpafiniti'
GP_user='att_dt_user'
GP_password=''
GP_Schema ='s_decision'
GP_port=6432
special_password =  urllib.parse.quote(GP_password)
conn_string = 'postgresql+psycopg2://'+GP_user+':'+special_password+'@'+GP_host+':'+str(GP_port)+'/'+GP_dbname
engine = create_engine(conn_string)



#Getting Max Date
max_available_date = pd.read_sql_query("SELECT COALESCE(max(acdr_calltime),'2022-07-01')::date as date FROM s_decision.artificial_dec_bm_on_acdr", conn)
startdate = max_available_date['date'][0]
Today_date =str(datetime.now().date())



while str(startdate)!=Today_date:
    conn = GP.connect(host=GP_host, dbname=GP_dbname, user=GP_user, password=GP_password, port=GP_port)
    enddate = startdate + timedelta(days=1)
    print("startdate:" +str(startdate))
    print("enddate:" +str(enddate))
    dataFrame       = pd.read_sql_query(f"""select guid, lob_id, call_time,benchmark::int as on_off from applied_ai_afiniti.t_acdr_main where call_time>='{str(startdate)}' ANd call_time<'{str(enddate)}' and Server='Mobility'""", conn)
    dataFrame['calltime_adj']=dataFrame.call_time.apply(lambda x: x + datetime.timedelta(hours=4))
    dataFrame['unix_time']=dataFrame.calltime_adj.apply(lambda x:int(x.timestamp()*1000))
    def bm_by_hash(ixn):
        value=ixn['unix_time']
        digest=hashlib.sha1(str(value).encode('utf-8')).hexdigest()
        digest_decimal=int(digest , 16)  % 10000
        return int(digest_decimal <=(33/85) * 10000)
                            
    dataFrame['hash_benchmark'] = dataFrame.apply(lambda x:0 if x['on_off']==0 else bm_by_hash(x),axis=1)
    cursor = conn.cursor()
    sql_Delete_query = f"""Delete from s_decision.artificial_dec_bm_on_acdr where acdr_calltime>='{str(startdate)}' ANd acdr_calltime<'{str(enddate)}'"""
    cursor.execute(sql_Delete_query)
    conn.commit()
    print('number of rows deleted', cursor.rowcount)
    q1 = """SELECT call_time as acdr_calltime, guid as callguid, lob_id, hash_benchmark as artificial_dec_bm FROM dataFrame 
    WHERE lob_id in (242,243,1012,183,12,219,414,34,41,54,64,85,101,1010,1011,154,173,177,1711,1713,182,2110,2119,2410,4413,-1,-2
    )"""
    df_insert = ps.sqldf(q1, locals())
    df_insert.to_sql('artificial_dec_bm_on_acdr', con = engine, schema=GP_Schema, if_exists = 'append', chunksize = 1000, index = False)
    conn.commit()
    conn.close()
    startdate = enddate


startdate:2022-08-23
enddate:2022-08-24
number of rows deleted 206784
startdate:2022-08-24
enddate:2022-08-25
number of rows deleted 0
startdate:2022-08-25
enddate:2022-08-26
number of rows deleted 0
startdate:2022-08-26
enddate:2022-08-27
number of rows deleted 0
startdate:2022-08-27
enddate:2022-08-28
number of rows deleted 0
startdate:2022-08-28
enddate:2022-08-29
number of rows deleted 0
startdate:2022-08-29
enddate:2022-08-30
number of rows deleted 0
startdate:2022-08-30
enddate:2022-08-31
number of rows deleted 0
startdate:2022-08-31
enddate:2022-09-01
number of rows deleted 0
startdate:2022-09-01
enddate:2022-09-02
number of rows deleted 0
startdate:2022-09-02
enddate:2022-09-03
number of rows deleted 0
startdate:2022-09-03
enddate:2022-09-04
number of rows deleted 0
startdate:2022-09-04
enddate:2022-09-05
number of rows deleted 0
startdate:2022-09-05
enddate:2022-09-06
number of rows deleted 0
startdate:2022-09-06
enddate:2022-09-07
number of rows deleted 0
startdate:2022-09-07

In [16]:
from datetime import datetime, timedelta
Today_date =str(datetime.now().date())
yesterday_date = (Today_date - timedelta(days=1)) 